## Name: Ahmed Mohamed Fekry Bagha
## ID: 4211032
## Group: (A)
## Section: (B1)

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

### As in Model_Loading_and_Training_(MobileNet) file

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU configured")
    except Exception as e:
        print("GPU configuration error:", e)
else:
    print("No GPU found, using CPU.")

IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 10
INITIAL_EPOCHS = 5  
FINE_TUNE_EPOCHS = 2 

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
subset_size = int(0.2 * x_train.shape[0])
x_train, y_train = x_train[:subset_size], y_train[:subset_size]

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    label = tf.one_hot(tf.squeeze(label), NUM_CLASSES)
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)\
                   .shuffle(1024)\
                   .batch(BATCH_SIZE)\
                   .prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)\
                 .batch(BATCH_SIZE)\
                 .prefetch(tf.data.AUTOTUNE)

base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

No GPU found, using CPU.


### Build the modified model

In [3]:
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)  
x = layers.Dropout(0.5)(x)                   
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
model = models.Model(inputs, outputs)

### Display model architecture

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               262400    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                2570  

### Set up callbacks

In [5]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True),
    ModelCheckpoint("modified_model.h5", monitor='val_loss', save_best_only=True)
]

### (1): Train only the newly added layers

In [6]:
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print("\nStarting initial training of modified model...\n")
model.fit(train_ds, epochs=INITIAL_EPOCHS, validation_data=test_ds, callbacks=callbacks, verbose=1)


Starting initial training of modified model...

Epoch 1/5
313/313 [==============================] - 262s 833ms/step - loss: 0.9632 - accuracy: 0.6710 - val_loss: 0.5319 - val_accuracy: 0.8206
Epoch 2/5
313/313 [==============================] - 267s 854ms/step - loss: 0.5989 - accuracy: 0.7969 - val_loss: 0.4795 - val_accuracy: 0.8383
Epoch 3/5
313/313 [==============================] - 269s 860ms/step - loss: 0.5235 - accuracy: 0.8190 - val_loss: 0.4751 - val_accuracy: 0.8354
Epoch 4/5
313/313 [==============================] - 270s 863ms/step - loss: 0.4735 - accuracy: 0.8370 - val_loss: 0.4556 - val_accuracy: 0.8421
Epoch 5/5
313/313 [==============================] - 274s 877ms/step - loss: 0.4365 - accuracy: 0.8470 - val_loss: 0.4422 - val_accuracy: 0.8515


### (2) Fine tuning (last 10 layers of the base model)

In [7]:
base_model.trainable = True
for layer in base_model.layers[:-10]:
    layer.trainable = False

### Recompile with a small learning rate

In [8]:
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print("\nStarting fine-tuning...\n")
model.fit(train_ds, epochs=INITIAL_EPOCHS + FINE_TUNE_EPOCHS,
          initial_epoch=INITIAL_EPOCHS, validation_data=test_ds, callbacks=callbacks, verbose=1)


Starting fine-tuning...

Epoch 6/7
313/313 [==============================] - 307s 976ms/step - loss: 0.6538 - accuracy: 0.7770 - val_loss: 0.6878 - val_accuracy: 0.7527
Epoch 7/7
313/313 [==============================] - 297s 948ms/step - loss: 0.5376 - accuracy: 0.8135 - val_loss: 0.5851 - val_accuracy: 0.7897


### Evaluate and final modified accuracy

In [10]:
loss, acc = model.evaluate(test_ds, verbose=0)
print(f"\nFinal Modified Model Test Accuracy: {acc*100:.2f}%")

Final Modified Model Test Accuracy: 85.11%
